# Ibtissam's LSTM-Attention Model

## **Acknowledgment**

- **Original source**: [*LSTM-Attention model.ipynb*](https://github.com/bibtissam/LSTM-Attention-FraudDetection/blob/main/LSTM-Attention%20model.ipynb)
- **Model's Article**: [*"Enhanced credit card fraud detection based on attention mechanism and LSTM deep model", 2021*](https://doi.org/10.1186/s40537-021-00541-8)

## Global Setting Variables

In [1]:
LASTEST_MODEL_NAME = 'model_2_Ibtissam_LSTM_acc'
MODEL_PATH = 'architectures/'
DATASET_PATH = 'dataset/'
NEW_MODEL = True
PERFORM_TRAINING = True
SAVE_MODEL = True
RANDOM_SEED = 42 # Set to `None` for the generator uses the current system time.

## Importing the necessary packages

In [2]:
# If you are running on `Binder`, then it is no need to set up the packages again
# %pip install -r requirements.txt

# ---OR---

# %pip install tensorflow==2.10.1 numpy==1.26.4 pandas scikit-learn imblearn matplotlib seaborn requests

In [3]:
# import pandas as pd
# import sklearn.metrics as metrique
# from pandas import Series
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from matplotlib import pyplot
# from sklearn.model_selection import train_test_split
# import numpy as np
# from keras.callbacks import EarlyStopping
# from keras.utils import np_utils
# from keras.callbacks import ModelCheckpoint
# from sklearn.metrics import accuracy_score
# from keras.models import Sequential
# from keras.utils import np_utils
# from keras.layers import RepeatVector, Dense, Activation, Lambda
# from keras.models import Sequential
# from keras import backend as K, regularizers, Model, metrics
# from keras.backend import cast

In [4]:
import pandas as pd
import numpy as np

import tensorflow as tf

import matplotlib.pyplot as plt
from tensorflow.keras.layers import Lambda, LSTM, Dense, Embedding, Dropout,Input, Attention, Layer, Concatenate, Permute, Dot, Multiply, Flatten
from tensorflow.keras import backend as K, regularizers, Model, metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import os
import utils
from collections import Counter
import itertools

np.random.seed(RANDOM_SEED)

Python Version: `3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]`
Base Python location: `C:\Users\LMT\AppData\Local\Programs\Python\Python310`
Current Environment location: `.venv_xai_fraud_detection`

Tensorflow version: `2.10.1`
CUDNN version: `64_8`
CUDA version: `64_112`
Num GPUs Available: 1


## Getting Relevant Data

### Download dataset

In [5]:
utils.download_dataset_from_kaggle('fraudTrain.csv')
utils.download_dataset_from_kaggle('fraudTest.csv')

URL: https://www.kaggle.com/api/v1/datasets/download/kartik2112/fraud-detection/fraudTrain.csv
File `dataset/fraudTrain.csv` already exists.
URL: https://www.kaggle.com/api/v1/datasets/download/kartik2112/fraud-detection/fraudTest.csv
File `dataset/fraudTest.csv` already exists.


### Read data

In [6]:
data_train = pd.read_csv(os.path.join(DATASET_PATH, 'fraudTrain.csv'), index_col=0)
data_test = pd.read_csv(os.path.join(DATASET_PATH, 'fraudTest.csv'), index_col=0)

## Credit Card Fraud Dataset Fields

| Field # | Field Name | Description |
|---------|------------|-------------|
| 1 | **trans_date_trans_time** | Date and time when transaction occurred |
| 2 | **cc_num** | Credit card number of customer |
| 3 | **merchant** | Name of merchant where transaction occurred |
| 4 | **category** | Category of merchant (e.g., retail, food, etc.) |
| 5 | **amt** | Amount of transaction |
| 6 | **first** | First name of credit card holder |
| 7 | **last** | Last name of credit card holder |
| 8 | **gender** | Gender of credit card holder |
| 9 | **street** | Street address of credit card holder |
| 10 | **city** | City of credit card holder |
| 11 | **state** | State of credit card holder |
| 12 | **zip** | ZIP code of credit card holder |
| 13 | **lat** | Latitude location of credit card holder |
| 14 | **long** | Longitude location of credit card holder |
| 15 | **city_pop** | Population of credit card holder's city |
| 16 | **job** | Occupation of credit card holder |
| 17 | **dob** | Date of birth of credit card holder |
| 18 | **trans_num** | Transaction number |
| 19 | **unix_time** | UNIX timestamp of transaction |
| 20 | **merch_lat** | Latitude location of merchant |
| 21 | **merch_long** | Longitude location of merchant |
| 22 | **is_fraud** | Target class indicating whether transaction is fraudulent (1) or legitimate (0) |

## Feature Engineering

In [7]:
# Generate and plot imbalanced classification dataset
# summarize class distribution
counter = Counter(data_train['is_fraud'])
print(counter)
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = np.where(data_train['is_fraud'] == label)[0]

Counter({0: 1289169, 1: 7506})


In [8]:
data_train = utils.feature_engineering(data_train)
data_test = utils.feature_engineering(data_test)

## Pre-processing

In [9]:
X_train, y_train, data_train, transformations_train = utils.pre_processing(data_train)

Ordinal-Encoding is applied for `['merchant', 'category', 'gender', 'age_group']`
SMOTE is applied


In [10]:
X_train.shape

(2578338, 13)

In [11]:
X_test, y_test, data_test, transformations_test = utils.pre_processing(data_test, isTestSet=True)

Ordinal-Encoding is applied for `['merchant', 'category', 'gender', 'age_group']`


In [12]:
X_test.shape

(555719, 13)

## Model Building

In [13]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [14]:
if NEW_MODEL:
    inputs = Input((X_train.shape[1],))  # Original 2D input (samples, features)
    reshaped = Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs)  # Add time dimension
    att_in_1=LSTM(50, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)(reshaped)
    att_in_2=LSTM(50, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)(att_in_1)
    att_out=attention()(att_in_2)
    outputs=Dense(1, activation='sigmoid', trainable=True)(att_out)
    model=Model(inputs, outputs, name='model_2_Ibtissam_LSTM')
else:
    model = utils.load_models(LASTEST_MODEL_NAME) # Continue training previous trained model if needed

## Training

In [ ]:
if PERFORM_TRAINING:
    epochs = 100
    batch_size=30000
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=4,
        verbose=1,
        mode='auto',
        baseline=None,
        restore_best_weights=True
    )
    
    history = model.fit(X_train, y_train, epochs=epochs,batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/100
86/86 [==============================] - 13s 79ms/step - loss: 0.5946 - accuracy: 0.7174 - val_loss: 0.4998 - val_accuracy: 0.9539
Epoch 2/100
86/86 [==============================] - 6s 70ms/step - loss: 0.4853 - accuracy: 0.7711 - val_loss: 0.4153 - val_accuracy: 0.9448
Epoch 3/100
86/86 [==============================] - 7s 86ms/step - loss: 0.4495 - accuracy: 0.7998 - val_loss: 0.3766 - val_accuracy: 0.9494
Epoch 4/100
86/86 [==============================] - 8s 97ms/step - loss: 0.4322 - accuracy: 0.8143 - val_loss: 0.3582 - val_accuracy: 0.9526
Epoch 5/100
86/86 [==============================] - 8s 94ms/step - loss: 0.4162 - accuracy: 0.8256 - val_loss: 0.3413 - val_accuracy: 0.9572
Epoch 6/100
86/86 [==============================] - 8s 91ms/step - loss: 0.3967 - accuracy: 0.8334 - val_loss: 0.3285 - val_accuracy: 0.9635
Epoch 7/100
86/86 [==============================] - 8s 98ms/step - loss: 0.3782 - accuracy: 0.8387 - val_loss: 0.3247 - val_accuracy: 0.9675
Epoch

## Learning Curve

In [ ]:
# plot loss during training
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
# plot accuracy during training
plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

## Predictions

In [ ]:
# predict probabilities for test set
y_predict = model.predict(X_test)
y_predict_binary = np.round(y_predict).astype(int).squeeze()

In [ ]:
acc_postfix = f'_acc{accuracy_score(y_test, y_predict_binary)*100:.0f}'

if model.name.find('_acc') != -1:
    model._name = model.name[:model.name.find('_acc')] + acc_postfix
else:
    model._name = model.name + acc_postfix

In [ ]:
# Save predictions
utils.save_predictions(model.name, y_predict)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
labels = ['Normal','Fraud']

plot_confusion_matrix(cm=confusion_matrix(y_true=y_test, y_pred=y_predict_binary), classes=labels, title='LSTM-Attention', normalize=False)

## Model Performance Metrics

In [ ]:
utils.get_model_metrics_df(y_test, y_predict)

## Save model

In [ ]:
if SAVE_MODEL:
    model.save(os.path.join(MODEL_PATH, model.name))
    # model.save(os.path.join(MODEL_PATH, model.name}.h5'))